In [1]:
import numpy as np, re
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, ReversibleField, TabularDataset, Iterator
from nltk.translate.bleu_score import sentence_bleu
import spacy
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support as score

import random
import math
import os

In [2]:
TWEET = Field()
TWEET_ID = Field(sequential=False, use_vocab=False)
LABEL = Field(sequential=False, use_vocab=False)

fields = {'tweet_50': ('t', TWEET), 'label_se': ('l', LABEL), 'tweet_id': ('id', TWEET_ID)}

train_data, valid_data, test_data = TabularDataset.splits(
                                        path = 'ArSAS/json',
                                        train = 'train_final.json',
                                        validation = 'dev_final.json',
                                        test = 'test_final.json',
                                        format = 'json',
                                        fields = fields
)


# for j in range(20000):
#     ob = vars(train_data[j])
#     if ob['id'] == 930347128425275392:
#         print(ob['t'])
#         break
print(vars(train_data[0]))    

TWEET.build_vocab(train_data)
# LABEL.build_vocab(train_data)
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), sort=False, batch_size= BATCH_SIZE, device=device)
print(type(train_iterator))



#     for j, elem in enumerate(z):
#         if elem.item() == 930347128425275392:
#             print("label ", y[j].item())
#             for r, index in enumerate(x):
#                 print(r, j)
#                 print(TWEET.vocab.itos[x[r][j]])
#             break
    
print('dev:')
for batch in valid_iterator:
    print(batch)
    break




{'t': ['وزير', 'خارجيه', 'فرنسا', 'عن', 'منتدي', 'شباب', 'العالم', 'شعرت', 'بارتياح', 'وانا', 'اتابعه', 'من', 'باريس', 'URL', 'HASH\n'], 'l': 0, 'id': 928615163250520065}
cpu
<class 'torchtext.data.iterator.BucketIterator'>
dev:

[torchtext.data.batch.Batch of size 128]
	[.t]:[torch.LongTensor of size 50x128]
	[.l]:[torch.LongTensor of size 128]
	[.id]:[torch.LongTensor of size 128]


In [3]:
class Attention(nn.Module):
    def __init__(self, query_dim, key_dim, value_dim):
        super(Attention, self).__init__()
        self.scale = 1. / math.sqrt(query_dim)

    def forward(self, query, keys, values):
        # Query = Hidden
        # hidden dim => (batch size, enc_hid_dim)
        # Query = [BxQ]
        # Keys = [TxBxK]
        # Values = [TxBxV]
        # Outputs = a:[TxB], lin_comb:[BxV]

        # Here we assume q_dim == k_dim (dot product attention)

        query = query.unsqueeze(1) # [BxQ] -> [Bx1xQ]
        keys = keys.transpose(0,1).transpose(1,2) # [TxBxK] -> [BxKxT]
        energy = torch.bmm(query, keys) # [Bx1xQ]x[BxKxT] -> [Bx1xT]
        energy = F.softmax(energy.mul_(self.scale), dim=2) # scale, normalize

        values = values.transpose(0,1) # [TxBxV] -> [BxTxV]
        linear_combination = torch.bmm(energy, values).squeeze(1) #[Bx1xT]x[BxTxV] -> [BxV]
        return energy, linear_combination



In [4]:
class Encoder(nn.Module):
    def __init__(self, attention, input_dim, emb_dim, enc_hid_dim, output_dim, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.attention=attention

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(enc_hid_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, input):
        
        # input dim => (src_sent_len, batch_size)
        embed = self.dropout(self.embedding(input))
        
        # embed dim => (src_sent_len, batch_size, emb_dim)
        outputs, hidden = self.rnn(embed)
        
        # outputs dim => (src_sent_len, batch size, enc_hid_dim * num directions)
        # hidden dim => (number of layers * number of directions, batch size, enc_hid_dim)
        hidden = torch.cat((hidden[-2, : , :], hidden[-1, :, :]), dim = 1)
        # hidden[-2, :, :] reduces 3d to 2d tensor since first dimension is now fixed, so dim = 1 is the last dimension
        # hidden dim => (batch size, enc_hid_dim)


        energy, attn_output = self.attention(hidden, outputs, outputs)
        predictions = self.linear(attn_output)
        
        
        
        return predictions

In [5]:
sentiment_labels = ['Positive', 'Negative', 'Neutral', 'Mixed']


INPUT_DIM = len(TWEET.vocab)
OUTPUT_DIM = len(sentiment_labels) - 1 # no mixed class
ENC_EMB_DIM = 256
ENC_HID_DIM = 512
ENC_DROPOUT = 0.5
ATTN_DIM = 500

attn = Attention(ATTN_DIM, ATTN_DIM, ATTN_DIM)
model = Encoder(attn, INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, OUTPUT_DIM, ENC_DROPOUT).to(device)


optimizer = optim.Adam(model.parameters())

# pad_idx = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss()




In [6]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    match_count = 0
    total_count = 0
    for i, batch in enumerate(iterator):
        print("train iterator ", i)
        src = batch.t
        trg = batch.l
#         print("src")
#         print(src)
#         print("target")
#         print(trg)
        optimizer.zero_grad()
        
        output = model(src)
#         print("output after seq2seq")
#         print(output)
#         print(output.shape)
        #trg = [sent len, batch size]
        #output = [sent len, batch size, output dim]
        
        #reshape to:
        #trg = [(sent len - 1) * batch size]
        #output = [(sent len - 1) * batch size, output dim]
        for j, ind_output in enumerate(output):
            max_index = ind_output.max(0)[1]
            if trg[j].item() == 3 and max_index.item() == 3:
                match_count += 1
            if trg[j].item() == 3:
                total_count += 1
    

        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
    print("match count ", match_count)   
    print("total count ", total_count)
        
    return epoch_loss / len(iterator)

In [7]:
def evaluate(model, iterator, criterion, testing):
    
    model.eval()
    epoch_loss = 0
    ground_truth, classification = [], []
    match_count = 0
    total_count = 0


    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.t
            trg = batch.l
            
            output = model(src) #turn off teacher forcing
            loss = criterion(output, trg)

            epoch_loss += loss.item()
            if not testing:
                for j, ind_output in enumerate(output):
                    max_index = ind_output.max(0)[1]
                    if trg[j].item() == 3 and max_index.item() == 3:
                        match_count += 1
                    if trg[j].item() == 3:
                        total_count += 1
                        
            if testing:
                
                for j, ind_output in enumerate(output):
                    max_index = ind_output.max(0)[1]
                    classification.append(max_index.item())
                    ground_truth.append(trg[j].item())
                    if ground_truth[-1] == 3:
                        print(ground_truth[-1], classification[-1])
                        
                    
    if testing:
        print("trg ", trg.shape)
        print("classification ", len(classification))
        precision, recall, fscore, support = score(np.array(ground_truth), classification)
        print("Detailed evaluation:")
        print('precision: {}'.format(precision))
        print('recall: {}'.format(recall))
        print('fscore: {}'.format(fscore))
        print('support: {}'.format(support))
    
    if not testing:
        print("dev set")
        print("match count ", match_count)   
        print("total count ", total_count)
        
    return epoch_loss / len(iterator)

In [8]:
N_EPOCHS = 20
CLIP = 0.001
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'se_weights_50.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(N_EPOCHS):
    print("epoch ", epoch)
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion, False)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:03} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f} |')
    print(f'| Epoch: {epoch+1:03} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}  |')

epoch  0
train iterator  0
train iterator  1
train iterator  2
train iterator  3
train iterator  4
train iterator  5
train iterator  6
train iterator  7
train iterator  8
train iterator  9
train iterator  10
train iterator  11
train iterator  12
train iterator  13
train iterator  14
train iterator  15
train iterator  16
train iterator  17
train iterator  18
train iterator  19
train iterator  20
train iterator  21
train iterator  22
train iterator  23
train iterator  24
train iterator  25
train iterator  26
train iterator  27
train iterator  28
train iterator  29
train iterator  30
train iterator  31
train iterator  32
train iterator  33
train iterator  34
train iterator  35
train iterator  36
train iterator  37
train iterator  38
train iterator  39
train iterator  40
train iterator  41
train iterator  42
train iterator  43
train iterator  44
train iterator  45
train iterator  46
train iterator  47
train iterator  48
train iterator  49
train iterator  50
train iterator  51
train iterato

train iterator  34
train iterator  35
train iterator  36
train iterator  37
train iterator  38
train iterator  39
train iterator  40
train iterator  41
train iterator  42
train iterator  43
train iterator  44
train iterator  45
train iterator  46
train iterator  47
train iterator  48
train iterator  49
train iterator  50
train iterator  51
train iterator  52
train iterator  53
train iterator  54
train iterator  55
train iterator  56
train iterator  57
train iterator  58
train iterator  59
train iterator  60
train iterator  61
train iterator  62
train iterator  63
train iterator  64
train iterator  65
train iterator  66
train iterator  67
train iterator  68
train iterator  69
train iterator  70
train iterator  71
train iterator  72
train iterator  73
train iterator  74
train iterator  75
train iterator  76
train iterator  77
train iterator  78
train iterator  79
train iterator  80
train iterator  81
train iterator  82
train iterator  83
train iterator  84
train iterator  85
train iterat

train iterator  68
train iterator  69
train iterator  70
train iterator  71
train iterator  72
train iterator  73
train iterator  74
train iterator  75
train iterator  76
train iterator  77
train iterator  78
train iterator  79
train iterator  80
train iterator  81
train iterator  82
train iterator  83
train iterator  84
train iterator  85
train iterator  86
train iterator  87
match count  0
total count  0
dev set
match count  0
total count  0
| Epoch: 009 | Train Loss: 0.405 | Train PPL:   1.499 | Val. Loss: 0.701 | Val. PPL:   2.015 |
| Epoch: 009 | Train Loss: 0.405 | Train PPL:   1.499  |
epoch  9
train iterator  0
train iterator  1
train iterator  2
train iterator  3
train iterator  4
train iterator  5
train iterator  6
train iterator  7
train iterator  8
train iterator  9
train iterator  10
train iterator  11
train iterator  12
train iterator  13
train iterator  14
train iterator  15
train iterator  16
train iterator  17
train iterator  18
train iterator  19
train iterator  20
tr

train iterator  1
train iterator  2
train iterator  3
train iterator  4
train iterator  5
train iterator  6
train iterator  7
train iterator  8
train iterator  9
train iterator  10
train iterator  11
train iterator  12
train iterator  13
train iterator  14
train iterator  15
train iterator  16
train iterator  17
train iterator  18
train iterator  19
train iterator  20
train iterator  21
train iterator  22
train iterator  23
train iterator  24
train iterator  25
train iterator  26
train iterator  27
train iterator  28
train iterator  29
train iterator  30
train iterator  31
train iterator  32
train iterator  33
train iterator  34
train iterator  35
train iterator  36
train iterator  37
train iterator  38
train iterator  39
train iterator  40
train iterator  41
train iterator  42
train iterator  43
train iterator  44
train iterator  45
train iterator  46
train iterator  47
train iterator  48
train iterator  49
train iterator  50
train iterator  51
train iterator  52
train iterator  53
tr

train iterator  35
train iterator  36
train iterator  37
train iterator  38
train iterator  39
train iterator  40
train iterator  41
train iterator  42
train iterator  43
train iterator  44
train iterator  45
train iterator  46
train iterator  47
train iterator  48
train iterator  49
train iterator  50
train iterator  51
train iterator  52
train iterator  53
train iterator  54
train iterator  55
train iterator  56
train iterator  57
train iterator  58
train iterator  59
train iterator  60
train iterator  61
train iterator  62
train iterator  63
train iterator  64
train iterator  65
train iterator  66
train iterator  67
train iterator  68
train iterator  69
train iterator  70
train iterator  71
train iterator  72
train iterator  73
train iterator  74
train iterator  75
train iterator  76
train iterator  77
train iterator  78
train iterator  79
train iterator  80
train iterator  81
train iterator  82
train iterator  83
train iterator  84
train iterator  85
train iterator  86
train iterat

In [9]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss = evaluate(model, test_iterator, criterion, True)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

trg  torch.Size([67])
classification  3651
Detailed evaluation:
precision: [0.62339332 0.76219105 0.73546512]
recall: [0.54926387 0.79901961 0.75522388]
fscore: [0.58398555 0.78017094 0.74521355]
support: [ 883 1428 1340]
| Test Loss: 0.684 | Test PPL:   1.981 |
